In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [2]:
import logging
import os
from datetime import datetime
import json
import numpy as np
import cv2
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, hamming_loss
from skmultilearn.adapt import MLkNN

# Logger setup
logging.basicConfig(filename='model_training_log.txt', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
# Function to redirect print statements to log
class Logger(object):
    def __init__(self, log_file):
        self.terminal = sys.stdout
        self.log = open(log_file, 'a')
    def write(self, message):
        print(message)
        self.terminal.write(message)
        self.log.write(message)
    def flush(self):
        self.terminal.flush()
        self.log.flush()

In [4]:
CACHE_PATH = '/kaggle/input/full-code-with-knn-cache'

In [5]:
import logging
import os
from datetime import datetime

def create_logger(size, feature_extract):
    log_filename = f"logs/{size[0]}_{feature_extract.__name__}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    os.makedirs(os.path.dirname(log_filename), exist_ok=True)
    logger = logging.getLogger(f"{feature_extract.__name__}_{size}")
    logger.setLevel(logging.INFO)
    file_handler = logging.FileHandler(log_filename)
    file_handler.setLevel(logging.INFO)
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    
    return logger

In [6]:
import shutil

In [7]:
DIR = '/kaggle/input/8-labels-cloth-classification'
IMG_DIR = os.path.join(DIR, 'imgs')
TEST_PATH = os.path.join(DIR, 'test', 'data.json')
TRAIN_PATH = os.path.join(DIR, 'train', 'data.json')
VAL_PATH = os.path.join(DIR, 'val', 'data.json')
CLASS_PATH = os.path.join(DIR, 'classes.txt')

SIZES = [64, 128, 224]
RESIZES = [(size, size) for size in SIZES]
FRACTION = 1

shutil.copy(CLASS_PATH, 'classes.txt')


'classes.txt'

In [8]:
import random

In [9]:
def extract_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features, _ = hog(
        gray_image,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm="L2-Hys",
        visualize=True,
    )
    return hog_features

In [10]:
def extract_edge_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)  # Gradient in x direction
    sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)  # Gradient in y direction
    sobel_edges = np.hypot(sobel_x, sobel_y)  # Compute the magnitude of gradients (edges)
    sobel_edges = np.uint8(np.absolute(sobel_edges))  # Convert to uint8 for display and further processing
    sobel_edges_flat = sobel_edges.flatten()
    return sobel_edges_flat

In [11]:
def extract_both_features(image):
    hog_features = extract_hog_features(image)
    edge_features = extract_edge_features(image)
    merged_features = np.concatenate((hog_features, edge_features))
    return merged_features

In [12]:
def load_data_from_json(json_file, img_dir, feature_extract, target_size=(224, 224)):
    with open(json_file, "r") as f:
        data = json.load(f)

    sample_data = random.sample(data, int(len(data) * FRACTION))

    X = []
    y = []

    for item in tqdm(sample_data, desc=f"Processing data"):
        img_path = item["imgPath"]
        label = item["labels"]  # Multi-hot encoded labels

        # Construct image path
        image_path = os.path.join(img_dir, img_path)

        if os.path.exists(image_path):
            image = cv2.imread(image_path)
            image = cv2.resize(image, target_size)

            # Extract features
            features = feature_extract(image)

            # Append features and labels
            X.append(features)
            y.append(label)

    return np.array(X), np.array(y)

In [13]:
def load_labels_from_file(label_file):
    with open(label_file, 'r') as f:
        labels = f.read().splitlines()
    return labels

In [14]:
labels = load_labels_from_file(CLASS_PATH)

In [15]:
import joblib
import os
import numpy as np
from sklearn.model_selection import GridSearchCV

In [16]:
CACHE = False
SAVED_DATA = True


def get_path_from_file_name(file_name: str):
    if CACHE_PATH is None:
        file_path = os.path.join(CACHE_PATH, file_name)
        if os.path.exists(file_path):
            return file_path
    if os.path.exists(file_name):
        return file_name
    return None


def load_or_process_data(file_name, logger, size, feature_extract, data_type, load_function):
    """
    Load data from cache or process and save it if not available.
    """
    path = get_path_from_file_name(file_name)
    if path and CACHE:
        logger.info(f"Loading preprocessed {data_type} data from {path}...")
        data = np.load(path)
        X, y = data[f'X_{data_type}'], data[f'y_{data_type}']
        logger.info(f"Loaded X_{data_type} shape: {X.shape}, y_{data_type} shape: {y.shape}")
    else:
        logger.info(f"Feature extract: {feature_extract.__name__}")
        logger.info(f"Resize: {size}")
        logger.info(f"Loading {data_type} data...")
        X, y = load_function()
        X = X.astype('float32')
        logger.info(f"X_{data_type} shape: {X.shape}, y_{data_type} shape: {y.shape}")

        if SAVED_DATA:
            logger.info(f"Saving preprocessed {data_type} data to {file_name}...")
            np.savez_compressed(file_name, **{f'X_{data_type}': X, f'y_{data_type}': y})

    return X, y


def load_combined_features(hog_file_name, edge_file_name, logger, size, data_type):
    """
    Load and combine HOG and Edge features if both are available.
    """
    hog_file_path = get_path_from_file_name(hog_file_name)
    edge_file_path = get_path_from_file_name(edge_file_name)
    if hog_file_path is None or edge_file_path is None:
        return None, None

    logger.info(f"Loading combined {data_type} features from {hog_file_path} and {edge_file_path}...")
    data_hog = np.load(hog_file_path)
    data_edge = np.load(edge_file_path)
    X_hog, y_hog = data_hog[f'X_{data_type}'], data_hog[f'y_{data_type}']
    X_edge = data_edge[f'X_{data_type}']
    X = np.concatenate((X_hog, X_edge), axis=1)
    logger.info(f"Combined X_{data_type} shape: {X.shape}, y_{data_type} shape: {y_hog.shape}")
    return X, y_hog


def generate_file_name(size: tuple[int], feature_extract, data_type):
    """
    Generate the file path for cached data.
    """
    file_name = f"data_{data_type}_{size[0]}_{feature_extract.__name__}_{FRACTION * 100}%.npz"
    return file_name

In [17]:
def process_data(size, feature_extract, data_type, logger):
    """
    Process data for the given feature extractor and data type.

    Args:
        size (tuple): Resize dimensions for the images.
        feature_extract (function): Feature extraction function.
        data_type (str): Type of data ('train' or 'test').
        logger (object): Logger for logging messages.

    Returns:
        tuple: Processed features and labels (X, y).
    """
    if feature_extract == extract_both_features:
        hog_file_name = generate_file_name(size, extract_hog_features, data_type)
        edge_file_name = generate_file_name(size, extract_edge_features, data_type)
        X, y = load_combined_features(hog_file_name, edge_file_name, logger, size, data_type)

        if X is None or y is None:
            combined_file_name = generate_file_name(size, extract_both_features, data_type)
            X, y = load_or_process_data(
                combined_file_name,
                logger,
                size,
                feature_extract,
                data_type,
                lambda: (
                    load_combined_data(data_type, feature_extract, size)
                )
            )
    else:
        file_name = generate_file_name(size, feature_extract, data_type)
        X, y = load_or_process_data(
            file_name,
            logger,
            size,
            feature_extract,
            data_type,
            lambda: (
                load_combined_data(data_type, feature_extract, size)
            )
        )
    return X, y


def load_combined_data(data_type, feature_extract, size):
    """
    Load combined data from the specified paths.

    Args:
        data_type (str): Type of data ('train' or 'test').
        feature_extract (function): Feature extraction function.
        size (tuple): Resize dimensions.

    Returns:
        tuple: Combined features and labels (X, y).
    """
    if data_type == "train":
        return (
            np.concatenate([
                load_data_from_json(TRAIN_PATH, IMG_DIR, feature_extract, size)[0],
                load_data_from_json(VAL_PATH, IMG_DIR, feature_extract, size)[0]
            ], axis=0),
            np.concatenate([
                load_data_from_json(TRAIN_PATH, IMG_DIR, feature_extract, size)[1],
                load_data_from_json(VAL_PATH, IMG_DIR, feature_extract, size)[1]
            ], axis=0)
        )
    return load_data_from_json(TEST_PATH, IMG_DIR, feature_extract, size)

In [18]:
# Main processing loop
for size in RESIZES:
    for feature_extract in [extract_hog_features, extract_edge_features, extract_both_features]:
        logger = create_logger(size, feature_extract)
        X_train, y_train = process_data(size, feature_extract, "train", logger)
        X_test, y_test = process_data(size, feature_extract, "test", logger)


2024-12-29 15:31:56,211 - INFO - Feature extract: extract_hog_features
2024-12-29 15:31:56,213 - INFO - Resize: (64, 64)
2024-12-29 15:31:56,214 - INFO - Loading train data...
Processing data: 100%|██████████| 4/4 [00:00<00:00, 32.60it/s]
2024-12-29 15:32:00,908 - INFO - X_train shape: (39, 1764), y_train shape: (39, 8)
2024-12-29 15:32:00,909 - INFO - Saving preprocessed train data to data_train_64_extract_hog_features_0.1%.npz...
2024-12-29 15:32:00,927 - INFO - Feature extract: extract_hog_features
2024-12-29 15:32:00,929 - INFO - Resize: (64, 64)
2024-12-29 15:32:00,930 - INFO - Loading test data...
Processing data: 100%|██████████| 4/4 [00:00<00:00, 30.99it/s]
2024-12-29 15:32:01,264 - INFO - X_test shape: (4, 1764), y_test shape: (4, 8)
2024-12-29 15:32:01,266 - INFO - Saving preprocessed test data to data_test_64_extract_hog_features_0.1%.npz...
2024-12-29 15:32:01,270 - INFO - Feature extract: extract_edge_features
2024-12-29 15:32:01,271 - INFO - Resize: (64, 64)
2024-12-29 15